In [ ]:
import pandas as pd 
import os

In [ ]:
"""split large datasets into smaller csv files
"""

# Parameters
input_csv = 'recipes_data.csv'  # Replace with the actual filename of your large CSV file
chunk_size = 100000  # Number of rows to read in each chunk
output_prefix = 'input_'  # Prefix for the output CSV filenames

# Initialize variables
chunk_number = 0

# Read the CSV file in chunks
for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    # Process the chunk as needed
    # ...
    
    # Save the chunk to a new CSV file
    output_filename = f"inputs/{output_prefix}{chunk_number}.csv"
    chunk.to_csv(output_filename, index=False)
    
    chunk_number += 1

In [ ]:
"""get list of inputs files to be processed
"""

directory = 'inputs/'  # Replace with the path to your directory

# List all file names in the directory
file_names = os.listdir(directory)



# list comprehsions 
file_names = [directory + filename for filename in file_names]



len(file_names)


In [5]:
output_file= 'outputs/recipes.txt'

In [14]:
"""process each csv file and format in a text description way suitable for trainig gpt neo model.
"""
for file_name in file_names:
    print(f"Processing : {file_name}")
    Recipes = pd.read_csv(file_name)
    Recipes.drop(columns=['source','NER','site','link'], inplace=True)
    columns= Recipes.columns
    
    with open(file_name.replace('input', 'output'), 'a') as f:
        for row in Recipes.itertuples():
            for column in columns:

                if column =='ingredients':
                    ingredients = getattr(row, column)[2:-2].replace('lasagne', 'lasagna').replace('1 qt.', '1 quart of').replace('qt.', 'quarts of').replace('1 c.', '1 cup of').replace('c.', 'cups of').replace('", "',', ').replace('lb.','pound').replace('lbs.','pounds').replace('oz.','ounces').replace('\\u00b0',' degrees Fahrenheit').replace('1 Tbsp.', '1 tablespoon of').replace('Tbsp.', 'tablespoons of').replace('1 tsp.', '1 teaspoon of').replace('tsp.', 'teaspoons of').replace('ingredients','Ingredients').replace('pkg.', 'package ').replace('","',',').split('","')
                    f.write(f"{column.upper()}: {','.join(ingredients)}; ")

                elif column =='directions':
                    #print(f" {column}: ".join(instructions))
                    instructions = getattr(row, column)[2:-2].replace('\\u00b0',' degrees Fahrenheit').replace('", "',', ').replace('.', ' ').split(' , ')
                    f.write(f"{column.upper()}: {'; '.join(instructions )}.")

                else:
                    #print(f"{'Recipe'}: {getattr(row, column)}, " + f"{'Ingredients'}: {','.join(ingredients)}, " + f"{'Directions'}: {', '.join(instructions)}.")
                    f.write(f"RECIPE: {getattr(row, column)}; ")

            f.write('\n\n')
                

Processing : inputs/input_0.csv
Processing : inputs/input_1.csv
Processing : inputs/input_10.csv
Processing : inputs/input_11.csv
Processing : inputs/input_12.csv
Processing : inputs/input_13.csv
Processing : inputs/input_14.csv
Processing : inputs/input_15.csv
Processing : inputs/input_16.csv
Processing : inputs/input_17.csv
Processing : inputs/input_18.csv
Processing : inputs/input_19.csv
Processing : inputs/input_2.csv
Processing : inputs/input_20.csv
Processing : inputs/input_21.csv
Processing : inputs/input_22.csv
Processing : inputs/input_3.csv
Processing : inputs/input_4.csv
Processing : inputs/input_5.csv
Processing : inputs/input_6.csv
Processing : inputs/input_7.csv
Processing : inputs/input_8.csv
Processing : inputs/input_9.csv


In [17]:
output_files = [x.replace('input','output') for x in file_names]



In [24]:

def check_token_size(file_data):
    """check for number of tokens

    Args:
        file_data (Stirng): file data read

    Returns:
        _type_: list of recipes
    """
    list_of_recipes = file_data.split('\n\n')
    return [x for x in list_of_recipes if len(x.split(' '))> 150 and len(x.split(' '))< 260]

In [25]:
"""filter recipes based on range of tokens within (120 and 260)
"""
filtered_list_of_recipes = []
for output_file in output_files:
    with open(output_file, 'r') as f:
        data = f.read()
        passed_recipes = check_token_size(data)
        print(f"file:{output_file} has {len(passed_recipes)} valid recipes.")
        filtered_list_of_recipes+=passed_recipes



print(len(filtered_list_of_recipes))



        
        

file:outputs/output_0.csv has 3989 valid recipes.
file:outputs/output_1.csv has 4004 valid recipes.
file:outputs/output_10.csv has 33699 valid recipes.
file:outputs/output_11.csv has 34122 valid recipes.
file:outputs/output_12.csv has 37874 valid recipes.
file:outputs/output_13.csv has 34383 valid recipes.
file:outputs/output_14.csv has 37929 valid recipes.
file:outputs/output_15.csv has 37519 valid recipes.
file:outputs/output_16.csv has 33338 valid recipes.
file:outputs/output_17.csv has 31448 valid recipes.
file:outputs/output_18.csv has 31527 valid recipes.
file:outputs/output_19.csv has 31613 valid recipes.
file:outputs/output_2.csv has 3802 valid recipes.
file:outputs/output_20.csv has 31491 valid recipes.
file:outputs/output_21.csv has 31469 valid recipes.
file:outputs/output_22.csv has 9694 valid recipes.
file:outputs/output_3.csv has 3911 valid recipes.
file:outputs/output_4.csv has 3841 valid recipes.
file:outputs/output_5.csv has 3792 valid recipes.
file:outputs/output_6.csv

In [27]:
"""write all the ouputs into a single text file
"""
with open('outputs/valid_recipes.txt', 'w') as f:
    for recipe in filtered_list_of_recipes[:100]:
        f.write(recipe)
        f.write('\n\n')